# Demo 03

In [2]:
import pandas as pd

import os
print(os.listdir("A:/Downloads/retinopathy-dataset-master"))
print(os.listdir("A:/Downloads/retinopathy-dataset-master/train"))

['dataset.csv', 'newTrainLabels.csv', 'README.md', 'retinopathy_solution.csv', 'test', 'train']
['train']


In [3]:
df = pd.read_csv("A:/Downloads/retinopathy-dataset-master/newTrainLabels.csv")
df.fillna(0, inplace=True)
df.head()

,image,level,MB
0,10_left,0,0.146893
1,10_right,0,0.134647
2,13_left,0,0.179653
3,13_right,0,0.173361
4,15_left,1,0.129906


In [4]:
# Create image data generator
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rescale=1./255, 
    validation_split = 0.25)

# Create data generators
train_generator = datagen.flow_from_dataframe(
    dataframe = df, 
    directory = "A:/Downloads/retinopathy-dataset-master/test/test/",
    has_ext = False,
    x_col = "image", 
    y_col = "level", 
    class_mode = "categorical", 
    target_size = (100, 100), 
    batch_size = 16,
    subset = 'training')

valid_generator = datagen.flow_from_dataframe(
    dataframe = df, 
    directory = "A:/Downloads/retinopathy-dataset-master/test/test/",
    has_ext = False,
    x_col = "image", 
    y_col = "level", 
    class_mode = "categorical", 
    target_size = (100, 100), 
    batch_size = 16,
    subset = 'validation')

Found 1548 images belonging to 5 classes.
Found 514 images belonging to 5 classes.


In [5]:
# Create a basic Sequential model with several Conv2D layers

from keras import Sequential
from keras.layers import Conv2D, Activation, MaxPooling2D, Dropout, Flatten, Dense
from keras import optimizers

model = Sequential()
# input: 100x100 images with 3 channels -> (100, 100, 3) tensors.
# this applies 32 convolution filters of size 3x3 each.
model.add(Conv2D(32, (3, 3), activation = 'relu', input_shape = (100, 100, 3)))
model.add(Conv2D(32, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation = 'relu'))
model.add(Conv2D(64, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation = 'softmax'))

# Try a custom metrics, needs to be calculated in backend (Tensorflow)  
from keras import backend
def rmse(y_true, y_pred):
    return backend.sqrt(backend.mean(backend.square(y_pred - y_true), axis=-1))

sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer = sgd,
             loss='categorical_crossentropy', 
              metrics = ["accuracy", "mse", rmse])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 98, 98, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 96, 96, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 48, 48, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 48, 48, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 46, 46, 64)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 44, 44, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 22, 22, 64)        0         
__________

In [6]:
# Calculate how many batches are needed to go through whole train and validation set
STEP_SIZE_TRAIN = train_generator.n // train_generator.batch_size
STEP_SIZE_VALID = valid_generator.n // valid_generator.batch_size
N = 10 # Number of epochs

# Train and count seconds
from time import time
t1 = time()
h = model.fit_generator(generator = train_generator,
                    steps_per_epoch = 50, #STEP_SIZE_TRAIN,
                    validation_data = valid_generator,
                    validation_steps = 15, #STEP_SIZE_VALID,
                    epochs = N,
                    verbose = 2)
t2 = time()
elapsed_time = (t2 - t1)

# Save the model
model.save('case2.h5')

# Print the total elapsed time and average time per epoch in format (hh:mm:ss)
from time import localtime, strftime
t_total = strftime('%H:%M:%S', localtime(t2 - t1))
t_per_e = strftime('%H:%M:%S', localtime((t2 - t1)/N))
print('Total elapsed time for {:d} epochs: {:s}'.format(N, t_total))
print('Average time per epoch:             {:s}'.format(t_per_e))

Epoch 1/10


FileNotFoundError: [Errno 2] No such file or directory: 'A:/Downloads/retinopathy-dataset-master/test/test/15118_left.jpeg'